In [25]:
#libraries used to extract, clean and manipulate the data
import pandas as pd
import numpy as np
import string
#To plot the graphs
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')
#library used to count the frequency of words
from sklearn.feature_extraction.text import CountVectorizer
#To create the sentiment analysis model, tokenization and lemmatization
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\annan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\annan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\annan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\annan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [26]:
import pandas as pd
df_not_clean = pd.read_csv('output.csv')
print(df_not_clean)

                                    song           artist       features  \
0                             Easy On Me            Adele            NaN   
1              STAY (with Justin Bieber)    The Kid LAROI  Justin Bieber   
2      INDUSTRY BABY (feat. Jack Harlow)        Lil Nas X    Jack Harlow   
3                             Fancy Like     Walker Hayes            NaN   
4                             Bad Habits       Ed Sheeran            NaN   
...                                  ...              ...            ...   
29675                               Judy  Frankie Vaughan            NaN   
29676                                NaN              NaN            NaN   
29677                                NaN              NaN            NaN   
29678                                NaN              NaN            NaN   
29679                                NaN              NaN            NaN   

                     track_id release_date release_date_precision  \
0      0gplL1WMoJ6

In [27]:
def clean_lyrics(df,column):
    """
    This function cleans the words without importance and fix the format of the  dataframe's column lyrics
    parameters:
    df = dataframe
    column = name of the column to clean
    """

    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
    df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
    df[column] = df[column].str.replace(r"\\n"," ")
    df[column] = df[column].str.replace(r"[^\w\d'\s]+","")
    df[column] = df[column].str.strip()
    return df

In [28]:
df = clean_lyrics(df_not_clean,'lyrics')
#Create the decades column
#Filter  data to use songs that have lyrics.
df = df[df['lyrics'].notnull()]
#Save the data into a csv file
df['lyrics'].head(10)

C:\Users\annan\AppData\Local\Temp\ipykernel_19068\889759071.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
C:\Users\annan\AppData\Local\Temp\ipykernel_19068\889759071.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
C:\Users\annan\AppData\Local\Temp\ipykernel_19068\889759071.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
C:\Users\annan\AppData\Local\Temp\ipykernel_19068\889759071.py:12: FutureWarning:

0    there ain't no gold in this river that i've be...
1    the kid laroi i do the same thing i told you t...
2    lil nas x  dddaytrip took it to ten hey  baby ...
3    ayy my girl is bangin' she's so low maintenanc...
4    one two three four ooh ooh  every time you com...
5    right said fred  i'm too sexy for my shirt too...
6    i took an arrow to the heart i never kissed a ...
7    ah  well good for you i guess you moved on rea...
8    yeah  wanna know what it's like like baby show...
9    if you wanna run away with me i know a galaxy ...
Name: lyrics, dtype: object

In [29]:
def unique(list1):
   # intilize a null list
     unique_list = []
   # traverse for all elements
     for x in list1:
         # check if exists in unique_list or not
         if x not in unique_list:
              unique_list.append(x)
     return unique_list

In [30]:
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords


def lyrics_to_words(document):
    """
    This function splits the text of lyrics to  single words, removing stopwords and doing the lemmatization to each word
    parameters:
    document: text to split to single words
    """
    stop_words = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stopwordremoval = " ".join([i for i in document.lower().split() if i not in stop_words])
    punctuationremoval = ''.join(ch for ch in stopwordremoval if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punctuationremoval.split())
    return normalized

In [31]:
#Stores unique words of each lyrics song into a new column called words
#list used to store the words
words = []
#iterate trought each lyric and split unique words appending the result into the words list
df = df.reset_index(drop=True)
for word in df['lyrics'].tolist():
    words.append(unique(lyrics_to_words(word).split()))
#create the new column with the information of words lists
df['words'] = words
df.head(10)

,song,artist,features,track_id,release_date,release_date_precision,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,lyrics,popularity,peak-rank,words
0,Easy On Me,Adele,NaN,0gplL1WMoJ6iYaPgMCL0gX,10/14/2021,day,0.604,0.366,5.0,-7.519,...,0.000000,0.1330,0.130,141.981,4.0,224695.0,there ain't no gold in this river that i've be...,85.0,1,"[aint, gold, river, ive, washin, hand, forever..."
1,STAY (with Justin Bieber),The Kid LAROI,Justin Bieber,5HCyWlXZPP0y6Gqq8TgA20,7/9/2021,day,0.591,0.764,1.0,-5.484,...,0.000000,0.1030,0.478,169.928,4.0,141806.0,the kid laroi i do the same thing i told you t...,89.0,1,"[kid, laroi, thing, told, never, would, id, ch..."
2,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,Jack Harlow,27NovPIUIRrOZoCHxABJwK,7/23/2021,day,0.736,0.704,3.0,-7.409,...,0.000000,0.0501,0.894,149.995,4.0,212000.0,lil nas x dddaytrip took it to ten hey baby ...,86.0,1,"[lil, na, x, dddaytrip, took, ten, hey, baby, ..."
3,Fancy Like,Walker Hayes,NaN,58UKC45GPNTflCN6nwCUeF,1/21/2022,day,0.647,0.765,1.0,-6.459,...,0.000000,0.3150,0.855,79.994,4.0,161853.0,ayy my girl is bangin' she's so low maintenanc...,78.0,3,"[ayy, girl, bangin, low, maintenance, need, ch..."
4,Bad Habits,Ed Sheeran,NaN,3rmo8F54jFF8OgYsqTxm5d,10/29/2021,day,0.807,0.893,11.0,-3.745,...,0.000028,0.3660,0.537,126.011,4.0,230747.0,one two three four ooh ooh every time you com...,89.0,2,"[one, two, three, four, ooh, every, time, come..."
5,Way 2 Sexy (with Future & Young Thug),Drake,"Future,Young Thug",0k1WUmIRnG3xU6fvvDVfRG,9/3/2021,day,0.803,0.597,11.0,-6.035,...,0.000005,0.3230,0.331,136.008,4.0,257605.0,right said fred i'm too sexy for my shirt too...,81.0,1,"[right, said, fred, im, sexy, shirt, hurt, mil..."
6,Shivers,Ed Sheeran,NaN,50nfwKoDiSYg8zOCREWAm5,10/29/2021,day,0.788,0.859,2.0,-2.724,...,0.000000,0.0424,0.822,141.020,4.0,207853.0,i took an arrow to the heart i never kissed a ...,88.0,7,"[took, arrow, heart, never, kissed, mouth, tas..."
7,good 4 u,Olivia Rodrigo,NaN,4ZtFanR9U6ndgddUvNcjcG,5/21/2021,day,0.563,0.664,9.0,-5.044,...,0.000000,0.0849,0.688,166.928,4.0,178147.0,ah well good for you i guess you moved on rea...,88.0,1,"[ah, well, good, guess, moved, really, easily,..."
8,Need to Know,Doja Cat,NaN,3Vi5XqYrmQgOYBajMWSvCi,6/25/2021,day,0.664,0.609,1.0,-6.509,...,0.000000,0.0926,0.194,130.041,4.0,210560.0,yeah wanna know what it's like like baby show...,84.0,9,"[yeah, wanna, know, like, baby, show, really, ..."
9,Levitating (feat. DaBaby),Dua Lipa,DaBaby,5nujrmhLynf4yMoMtj8AQF,3/27/2020,day,0.702,0.825,6.0,-3.787,...,0.000000,0.0674,0.915,102.977,4.0,203064.0,if you wanna run away with me i know a galaxy ...,85.0,2,"[wanna, run, away, know, galaxy, take, ride, p..."


In [32]:
#Create lists to store the different scores for each word
negative = []
neutral = []
positive = []
compound = []
#Initialize the model
sid = SentimentIntensityAnalyzer()
#Iterate for each row of lyrics and append the scores
for i in df.index:
    scores = sid.polarity_scores(df['lyrics'].iloc[i])
    negative.append(scores['neg'])
    neutral.append(scores['neu'])
    positive.append(scores['pos'])
    compound.append(scores['compound'])
#Create 4 columns to the main data frame  for each score
df['negative'] = negative
df['neutral'] = neutral
df['positive'] = positive
df['compound'] = compound
df.head(10)

,song,artist,features,track_id,release_date,release_date_precision,danceability,energy,key,loudness,...,time_signature,duration_ms,lyrics,popularity,peak-rank,words,negative,neutral,positive,compound
0,Easy On Me,Adele,NaN,0gplL1WMoJ6iYaPgMCL0gX,10/14/2021,day,0.604,0.366,5.0,-7.519,...,4.0,224695.0,there ain't no gold in this river that i've be...,85.0,1,"[aint, gold, river, ive, washin, hand, forever...",0.043,0.745,0.211,0.9888
1,STAY (with Justin Bieber),The Kid LAROI,Justin Bieber,5HCyWlXZPP0y6Gqq8TgA20,7/9/2021,day,0.591,0.764,1.0,-5.484,...,4.0,141806.0,the kid laroi i do the same thing i told you t...,89.0,1,"[kid, laroi, thing, told, never, would, id, ch...",0.105,0.820,0.074,-0.8230
2,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,Jack Harlow,27NovPIUIRrOZoCHxABJwK,7/23/2021,day,0.736,0.704,3.0,-7.409,...,4.0,212000.0,lil nas x dddaytrip took it to ten hey baby ...,86.0,1,"[lil, na, x, dddaytrip, took, ten, hey, baby, ...",0.028,0.753,0.219,0.9921
3,Fancy Like,Walker Hayes,NaN,58UKC45GPNTflCN6nwCUeF,1/21/2022,day,0.647,0.765,1.0,-6.459,...,4.0,161853.0,ayy my girl is bangin' she's so low maintenanc...,78.0,3,"[ayy, girl, bangin, low, maintenance, need, ch...",0.016,0.686,0.298,0.9966
4,Bad Habits,Ed Sheeran,NaN,3rmo8F54jFF8OgYsqTxm5d,10/29/2021,day,0.807,0.893,11.0,-3.745,...,4.0,230747.0,one two three four ooh ooh every time you com...,89.0,2,"[one, two, three, four, ooh, every, time, come...",0.197,0.723,0.080,-0.9671
5,Way 2 Sexy (with Future & Young Thug),Drake,"Future,Young Thug",0k1WUmIRnG3xU6fvvDVfRG,9/3/2021,day,0.803,0.597,11.0,-6.035,...,4.0,257605.0,right said fred i'm too sexy for my shirt too...,81.0,1,"[right, said, fred, im, sexy, shirt, hurt, mil...",0.034,0.502,0.464,0.9994
6,Shivers,Ed Sheeran,NaN,50nfwKoDiSYg8zOCREWAm5,10/29/2021,day,0.788,0.859,2.0,-2.724,...,4.0,207853.0,i took an arrow to the heart i never kissed a ...,88.0,7,"[took, arrow, heart, never, kissed, mouth, tas...",0.024,0.787,0.189,0.9901
7,good 4 u,Olivia Rodrigo,NaN,4ZtFanR9U6ndgddUvNcjcG,5/21/2021,day,0.563,0.664,9.0,-5.044,...,4.0,178147.0,ah well good for you i guess you moved on rea...,88.0,1,"[ah, well, good, guess, moved, really, easily,...",0.054,0.682,0.264,0.9968
8,Need to Know,Doja Cat,NaN,3Vi5XqYrmQgOYBajMWSvCi,6/25/2021,day,0.664,0.609,1.0,-6.509,...,4.0,210560.0,yeah wanna know what it's like like baby show...,84.0,9,"[yeah, wanna, know, like, baby, show, really, ...",0.095,0.727,0.178,0.9354
9,Levitating (feat. DaBaby),Dua Lipa,DaBaby,5nujrmhLynf4yMoMtj8AQF,3/27/2020,day,0.702,0.825,6.0,-3.787,...,4.0,203064.0,if you wanna run away with me i know a galaxy ...,85.0,2,"[wanna, run, away, know, galaxy, take, ride, p...",0.012,0.836,0.153,0.9722


In [33]:
df = df[df['release_date'].notna()]

print(df.shape)
df['release_date'] = pd.to_datetime(df['release_date']).dt.year
df = df.drop(columns = 'release_date_precision')
df.head()
df.to_csv('lyrics.csv',index=False)

(22125, 27)


In [34]:
# print(df.iloc[22123]['words'])

['doo', 'well', 'went', 'dance', 'night', 'oooh', 'everybody', 'stag', 'said', 'gonna', 'drag', 'aonce', 'happened', 'prettiest', 'world', 'please', 'awont', 'come', 'talk', 'anda', 'girl', 'thata', 'sorry', 'little', 'bit', 'late', 'would', 'wait', 'steady', 'date']


In [40]:
# #Create a new dataframe of all the  words used in lyrics and its decades
# #list used to store the information
# set_words = []
# set_years = []
# #Iterate trought each word and year and stores them into the new lists
# for index, row in df.iterrows():
#     for word in row['words']:
#         set_words.append(word)
#         set_years.append(row['release_date'])
# # print(len(set_words))
# # with open('check2.txt', 'w') as f:
# #     for item in set_words:
# #             # write each item on a new line
# #             f.write(item + "," + set_years + "\n" )
# # print("done")
# #create the new data frame  with the information of words and year lists
# words_df = pd.DataFrame({'words':set_words,'year':set_years})
# #Defined  your own Stopwords in case the clean data function does not remove all of them
# stop_words =[]
# # count the frequency of each word that aren't on the stop_words lists
# cv = CountVectorizer(stop_words=stop_words)
# #Create a dataframe called data_cv to store the the number of times the word was used in  a lyric based their decades
# text_cv = cv.fit_transform(words_df['words'].iloc[:])
# data_cv = pd.DataFrame(text_cv.astype('int8').toarray(),columns=cv.get_feature_names())
# data_cv['year'] = words_df['year']
# #created a dataframe that Sums the ocurrence frequency of each word and group the result by year
# vect_words = data_cv.groupby('year').sum().T
# vect_words = vect_words.reset_index(level=0).rename(columns ={'index':'words'})
# vect_words = vect_words.rename_axis(columns='')
# #Save the data into a csv file
# vect_words.to_csv('words.csv',index=False)
# #change the order of columns to order from the oldest to actual year

MemoryError: Unable to allocate 64.5 GiB for an array with shape (1406803, 49197) and data type int8